In [2]:
!pip install nltk
!pip install rouge-score
!pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import sacrebleu
import subprocess

import nltk

# Download necessary NLTK resources
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /home/alexm/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alexm/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
dataset = pd.read_csv('data/code_completion_output.csv')

In [31]:
dataset.head()

,file_path,prefix,middle,suffix,generated_code
0,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,if len(parsed_prices) == 1:\n retur...,"def process_billa(price_str, price_type):\n ...",\n\n # Logic to compare against previous fo...
1,../PycharmProjects/sales_telegram_bot/backend/...,import json\nimport os\nfrom flask import Flas...,image_file = request.files['image']\n ...,torch.cuda.empty_cache() # Clear CUDA...,
2,../airflow/dags/data_pipeline.py,import json\nimport ast\nimport logging\nimpor...,trie = Trie()\n for word in flat_words:...,try:\n detection_data = ast.literal...,trie = Trie()\n\n # Run YOLO Model 2 on...
3,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,"return {""item_price"": item_price, ""ini...","return {""item_price"": parsed_prices[0]...",parsed_prices = [parse_price(p) for p ...
4,../airflow/dags/utils/yolo_ocr_utils.py,import json\nimport os\nimport tempfile\n\nimp...,"with open(image_path, 'rb') as image_f...","return data.get('extracted_text', ...",elif response.status_code == 200:\n ...


## Manual Evaluation

I now want to manually evaluate the generated code. I will rate the visual 'quality' and 'similarity' of the code with scores of 0, 0.5, or 1. A score of 0 indicates no similarity, 0.5 indicates some similarities, and 1 indicates that the code is almost correct or matches exactly.

In [12]:
# Placeholder for manual scores
manual_scores = []

# Iterate over each row in the dataset
for idx, row in dataset.iterrows():
    print(f"Instance {idx + 1}")
    print("File Path:", row['file_path'])
    
    # Display Actual Code (middle) and Generated Code
    actual_code = row['middle']
    generated_code = row['generated_code']
    
    print("\nActual Code (middle):\n", actual_code)
    print("\nGenerated Code:\n", generated_code)
    
    # Prompt for manual score input
    score = input("Enter your score for similarity (0 = no similarity, 0.5 = some similarity, 1 = almost correct or exact): ")
    
    # Append score to list
    try:
        score = float(score)
        if score in {0, 0.5, 1}:
            manual_scores.append(score)
        else:
            print("Invalid input. Score must be 0, 0.5, or 1.")
            break
    except ValueError:
        print("Invalid input. Please enter a numeric score of 0, 0.5, or 1.")
        break
    
    # Divider for readability
    print("-" * 80, "\n")

# Add scores to the dataset
dataset['manual_score'] = manual_scores

# Save the dataset with scores for later analysis
dataset.to_csv('data/evaluated_code_completion_output.csv', index=False)

Instance 1
File Path: ../airflow/dags/utils/price_processing.py

Actual Code (middle):
     if len(parsed_prices) == 1:
        return {"item_price": parsed_prices[0]}

    return None


# Billa Condition


Generated Code:
 

    # Logic to compare against previous formatting (price was less than current price)
    if packaging is not None:
        new_price = packaging[: packaging.index('.')]  # Add the packaging index before replacement
        old_price = packaging[packaging.index('.') + 1:-1].strip()

        if old_price!= new_price:
            return False

    return True


# New York Condition
def process_newyork(price_str):
    return parse_price(price_str)


# Tango Condition
def process_tango(price_str):
    return parse_price


Enter your score for similarity (0 = no similarity, 0.5 = some similarity, 1 = almost correct or exact):  0


-------------------------------------------------------------------------------- 

Instance 2
File Path: ../PycharmProjects/sales_telegram_bot/backend/models_app/app.py

Actual Code (middle):
         image_file = request.files['image']
        image_file.save(temp_img_file.name)
        temp_img_path = temp_img_file.name

    try:
        # Extract text using GOT-OCR2_0 model's chat interface
        extracted_text = extract_text_from_image(temp_img_path)

        return jsonify({'extracted_text': extracted_text}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

    finally:
        # Clean up: Delete the temporary image file
        os.remove(temp_img_path)


Generated Code:
    


Enter your score for similarity (0 = no similarity, 0.5 = some similarity, 1 = almost correct or exact):  0


-------------------------------------------------------------------------------- 

Instance 3
File Path: ../airflow/dags/data_pipeline.py

Actual Code (middle):
     trie = Trie()
    for word in flat_words:
        trie.insert(word)

    # Parse detection_data string to dictionary


Generated Code:
     trie = Trie()

    # Run YOLO Model 2 on images and perform OCR



...

-------------------------------------------------------------------------------- 

Instance 49
File Path: ../airflow/dags/utils/pdf_utils.py

Actual Code (middle):
         page_path = os.path.join(TEMP_DIR, page_filename)

        # Save the image locally
        image.save(page_path, 'PNG')

        # Upload each page to S3 in the 'pages/valid/' directory
        s3_page_path = f'{PAGES_S3_PATH}/{page_filename}'
        upload_file_to_s3(page_path, s3_page_path)

        # Add full S3 path of the page to the list
        page_s3_paths.append(s3_page_path)

    # Return the list of full S3 paths for the uploaded pages


Generated Code:
         # Upload the page to S3
        file_upload = upload_file_to_s3(
            file_path=file_path, image_data=image['Contents'][0], shop_name=shop_name,
            page_id='{}'.format(i),
            content="{} {}".format(image["Content"]["PageMetadata"], image["Content"]["Body"])))

        # Save page's PDF to S3
        page_s3_paths.append

Enter your score for similarity (0 = no similarity, 0.5 = some similarity, 1 = almost correct or exact):  1


-------------------------------------------------------------------------------- 

Instance 50
File Path: ../airflow/dags/utils/correct_names.py

Actual Code (middle):
         variants = generate_il1rjeo_combinations(word)


Generated Code:
         """
        Inserts a word into the Trie. If it's already present in the Trie, then replaces this word with another one.
        Words should be lower cased, separated by commas, and no spaces or punctuation.

        :param word: The word to add
        :type word: str
        :return: None
        :rtype: None
        """
        if not word:
            raise ValueError("Word cannot be empty.")
        if word[-2:] == ",":
            word += ","
        try:
            word = int(word)
        except TypeError:
            pass

        if word < 0:
            raise


Enter your score for similarity (0 = no similarity, 0.5 = some similarity, 1 = almost correct or exact):  0


-------------------------------------------------------------------------------- 



In [24]:
eval_df = pd.read_csv('evaluated_code_completion_output.csv')

In [25]:
eval_df.head()

,file_path,prefix,middle,suffix,generated_code,manual_score
0,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,if len(parsed_prices) == 1:\n retur...,"def process_billa(price_str, price_type):\n ...",\n\n # Logic to compare against previous fo...,0.0
1,../PycharmProjects/sales_telegram_bot/backend/...,import json\nimport os\nfrom flask import Flas...,image_file = request.files['image']\n ...,torch.cuda.empty_cache() # Clear CUDA...,,0.0
2,../airflow/dags/data_pipeline.py,import json\nimport ast\nimport logging\nimpor...,trie = Trie()\n for word in flat_words:...,try:\n detection_data = ast.literal...,trie = Trie()\n\n # Run YOLO Model 2 on...,0.5
3,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,"return {""item_price"": item_price, ""ini...","return {""item_price"": parsed_prices[0]...",parsed_prices = [parse_price(p) for p ...,0.0
4,../airflow/dags/utils/yolo_ocr_utils.py,import json\nimport os\nimport tempfile\n\nimp...,"with open(image_path, 'rb') as image_f...","return data.get('extracted_text', ...",elif response.status_code == 200:\n ...,0.0


## Automatic evaluation

The script under evaluates the generated code completions using six metrics that capture syntactic and semantic similarity between generated and reference code. The following metrics are included:
* Exact Match: Checks if the generated code exactly matches the reference code, providing a binary evaluation of precision.
* BLEU Score: Measures the overlap of n-grams, with higher scores indicating better exact matching of phrases.
* ROUGE-L: Focuses on the longest common subsequence, rewarding sequences that capture similar phrases or structural patterns.
* METEOR: Considers synonyms and stemming, enhancing evaluation for semantically similar but syntactically different phrases.
* ChrF: A character-level metric that rewards structural similarity, ideal for comparing code with potential variations in variable names.

In [26]:
# Initialize lists to store metric results
exact_matches = []
bleu_scores = []
rouge_l_scores = []
meteor_scores = []
chrf_scores = []

# Calculate metrics for each row, comparing only the middle part for all except ChrF
for index, row in eval_df.iterrows():
    # Reference and generated middle parts
    reference_middle = row['middle'].strip()
    generated_code = row['generated_code'].strip()
    
    # Find the generated middle part by locating the first instance of suffix
    suffix = row['suffix'].strip()
    if suffix in generated_code:
        generated_middle = generated_code.split(suffix)[0].strip()
    else:
        generated_middle = generated_code

    # Exact Match
    exact_match = 1.0 if reference_middle == generated_middle else 0.0
    exact_matches.append(exact_match)

    # BLEU Score with smoothing
    bleu_score = sentence_bleu([reference_middle.split()], generated_middle.split(), smoothing_function=SmoothingFunction().method1)
    bleu_scores.append(bleu_score)

    # ROUGE-L Score
    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = rouge.score(reference_middle, generated_middle)['rougeL'].fmeasure
    rouge_l_scores.append(rouge_l)

    # METEOR Score
    meteor = meteor_score([word_tokenize(reference_middle)], word_tokenize(generated_middle))
    meteor_scores.append(meteor)

    # ChrF Score using full reference and generated code
    reference_full = f"{row['prefix'].strip()} {reference_middle} {suffix}"
    generated_full = f"{row['prefix'].strip()} {generated_middle} {suffix}"
    chrf_score = sacrebleu.corpus_chrf([reference_full], [generated_full])
    chrf_scores.append(chrf_score.score)

# Add metric columns to the DataFrame
eval_df['Exact Match'] = exact_matches
eval_df['BLEU'] = bleu_scores
eval_df['ROUGE-L'] = rouge_l_scores
eval_df['METEOR'] = meteor_scores
eval_df['ChrF'] = chrf_scores

# Save updated DataFrame to a new CSV
output_path = 'data/evaluated_code_completion_output_with_metrics.csv'
eval_df.to_csv(output_path, index=False)

## Manual and Automatic evaluation comparison

In [27]:
full_eval_df = pd.read_csv('data/evaluated_code_completion_output_with_metrics.csv')

In [28]:
full_eval_df.head(50)

,file_path,prefix,middle,suffix,generated_code,manual_score,Exact Match,BLEU,ROUGE-L,METEOR,ChrF
0,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,if len(parsed_prices) == 1:\n retur...,"def process_billa(price_str, price_type):\n ...",\n\n # Logic to compare against previous fo...,0.0,0.0,0.004880,0.192771,0.254438,0.045090
1,../PycharmProjects/sales_telegram_bot/backend/...,import json\nimport os\nfrom flask import Flas...,image_file = request.files['image']\n ...,torch.cuda.empty_cache() # Clear CUDA...,,0.0,0.0,0.000000,0.000000,0.000000,0.083361
2,../airflow/dags/data_pipeline.py,import json\nimport ast\nimport logging\nimpor...,trie = Trie()\n for word in flat_words:...,try:\n detection_data = ast.literal...,trie = Trie()\n\n # Run YOLO Model 2 on...,0.5,0.0,0.076513,0.148148,0.288462,0.057110
3,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,"return {""item_price"": item_price, ""ini...","return {""item_price"": parsed_prices[0]...",parsed_prices = [parse_price(p) for p ...,0.0,0.0,0.009207,0.285714,0.209894,0.045090
4,../airflow/dags/utils/yolo_ocr_utils.py,import json\nimport os\nimport tempfile\n\nimp...,"with open(image_path, 'rb') as image_f...","return data.get('extracted_text', ...",elif response.status_code == 200:\n ...,0.0,0.0,0.123396,0.127273,0.302807,0.077184
5,../PycharmProjects/sales_telegram_bot/backend/...,# Copyright (c) 2010-2020 Benjamin Peterson\n#...,for attr in _urllib_request_moved_attributes:\...,del attr\n\nModule_six_moves_urllib_request._m...,"args, **kwargs)\n\n\nif PY3:\n def getsetde...",0.0,0.0,0.000000,0.029412,0.121951,0.018698
6,../airflow/dags/utils/s3_dynamodb_utils.py,import boto3\n\n# Initialize AWS S3 and Dynamo...,"print(f""Updated item in table {table_n...","print(f""Error updating item in DynamoD...",except Exception as e:\n,1.0,0.0,0.135335,0.421053,0.234906,0.259336
7,../airflow/dags/utils/correct_names.py,import re\nimport hunspell\nimport itertools\n...,for start in range(len(text)):\n ...,char = text[end]\n ...,start = 0\n end = len(text)\n\n...,0.0,0.0,0.002770,0.148936,0.261339,0.130141
8,../airflow/dags/utils/price_processing.py,import re\n\n\n# Helper function to parse pric...,"prices = re.findall(r'\d+[.,]?\d*', price_...",parsed_prices = [parse_price(p) for p in p...,"prices = re.findall(r'\d+[.,]?\d*', price_...",1.0,0.0,0.098493,0.380952,0.747174,0.045090
9,../airflow/dags/utils/correct_names.py,import re\nimport hunspell\nimport itertools\n...,variants.append(''.join(modified_word)...,class TrieNode:\n def __init__(self):\n ...,for position in positions:\n ...,1.0,0.0,0.033064,0.363636,0.277094,0.130141


In [29]:
# Select relevant columns for correlation analysis
metrics = ['Exact Match', 'BLEU', 'ROUGE-L', 'METEOR', 'ChrF']
correlations = {}

# Calculate Spearman and Kendall correlations for each metric with manual score
for metric in metrics:
    spearman_corr = eval_df['manual_score'].corr(eval_df[metric], method='spearman')
    kendall_corr = eval_df['manual_score'].corr(eval_df[metric], method='kendall')
    
    # Store results
    correlations[metric] = {
        'Spearman': spearman_corr,
        'Kendall': kendall_corr
    }

# Convert to DataFrame for better visualization
correlation_df = pd.DataFrame(correlations).T
print("Correlation between each metric and manual score:")
print(correlation_df)

Correlation between each metric and manual score:
             Spearman   Kendall
Exact Match       NaN       NaN
BLEU         0.613213  0.497908
ROUGE-L      0.612819  0.494840
METEOR       0.487628  0.379940
ChrF         0.424102  0.351727


/usr/lib/python3/dist-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


## Conclusion

The correlation results indicate that BLEU and ROUGE-L have the strongest relationships with the manual scores, as reflected in both Spearman and Kendall correlation coefficients (0.613 and 0.497 for BLEU, and 0.612 and 0.495 for ROUGE-L, respectively). This suggests that BLEU and ROUGE-L are more aligned with human judgment in evaluating the similarity of generated code completions, particularly in cases where exact structural and lexical match plays a role.